In [1]:
# organize imports
import os
import glob
import datetime

In [2]:
# print start time
print ("[INFO] program started on - " + str(datetime.datetime.now))

[INFO] program started on - <built-in method now of type object at 0x000000006BD22400>


In [3]:
# get the input and output path
input_path  = "H:\\projects\\logos32\\logo"
output_path = "H:\\projects\\logos32\\dataset\\train"

### Create conf/conf.json

In [ ]:
{
  "model"           : "inceptionv3",
  "weights"         : "imagenet",
  "include_top"     : false,

  "train_path"      : "dataset/train",
  "test_path"       : "dataset/test",
  "features_path"   : "output/logo32/inceptionv3/features.h5",
  "labels_path"     : "output/logo32/inceptionv3/labels.h5",
  "results"         : "output/logo32/inceptionv3/results.txt",
  "classifier_path" : "output/logo32/inceptionv3/classifier.pickle",
  "model_path"      : "output/logo32/inceptionv3/model",

  "test_size"       : 0.10,
  "seed"            : 9,
  "num_classes"     : 32
}

In [4]:
# filter warnings
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [ ]:
## no such layer: flatten or custom
## This error is the common one found when trying to add Flatten() in any of the model’s .py file. Please update Keras to the latest version. Completely uninstall Keras and reinstall it using pip. Now, get into the similar directory shown below where you have installed Anaconda.
## Inside this directory, you can see all the pre-trained models .py file. If you use InceptionV3 as the model, then open inception_v3.py.
## from ..layers import Flatten
## include_top as False
 """"if include_top:
        # Classification block
        x = GlobalAveragePooling2D(name='avg_pool')(x)
        x = Dense(classes, activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)
        x = Flatten(name='custom')(x)"""

In [5]:
# keras imports
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.applications.xception import Xception, preprocess_input
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.models import model_from_json
from keras.layers import Input

Using TensorFlow backend.


In [6]:
# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
import glob
import cv2
import h5py
import os
import json
import datetime
import time

In [7]:
# load the user configs
with open('conf/conf.json') as f:    
  config = json.load(f)

In [8]:
# config variables
model_name    = config["model"]
weights     = config["weights"]
include_top   = config["include_top"]
train_path    = config["train_path"]
features_path   = config["features_path"]
labels_path   = config["labels_path"]
test_size     = config["test_size"]
results     = config["results"]
model_path    = config["model_path"]

In [9]:
# start time
print ("[STATUS] start time - {}".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")))
start = time.time()

[STATUS] start time - 2018-09-14 00:09


In [10]:
# create the pretrained models
# check for pretrained weight usage or not
# check for top layers to be included or not
if model_name == "vgg16":
  base_model = VGG16(weights=weights)
  model = Model(input=base_model.input, output=base_model.get_layer('fc1').output)
  image_size = (224, 224)
elif model_name == "vgg19":
  base_model = VGG19(weights=weights)
  model = Model(input=base_model.input, output=base_model.get_layer('fc1').output)
  image_size = (224, 224)
elif model_name == "resnet50":
  base_model = ResNet50(weights=weights)
  model = Model(input=base_model.input, output=base_model.get_layer('flatten').output)
  image_size = (224, 224)
elif model_name == "inceptionv3":
  base_model = InceptionV3(include_top=include_top, weights=weights, input_tensor=Input(shape=(299,299,3)))
  model = Model(input=base_model.input, output=base_model.get_layer('custom').output)
  image_size = (299, 299)
elif model_name == "inceptionresnetv2":
  base_model = InceptionResNetV2(include_top=include_top, weights=weights, input_tensor=Input(shape=(299,299,3)))
  model = Model(input=base_model.input, output=base_model.get_layer('custom').output)
  image_size = (299, 299)
elif model_name == "mobilenet":
  base_model = MobileNet(include_top=include_top, weights=weights, input_tensor=Input(shape=(224,224,3)), input_shape=(224,224,3))
  model = Model(input=base_model.input, output=base_model.get_layer('custom').output)
  image_size = (224, 224)
elif model_name == "xception":
  base_model = Xception(weights=weights)
  model = Model(input=base_model.input, output=base_model.get_layer('avg_pool').output)
  image_size = (299, 299)
else:
  base_model = None

print ("[INFO] successfully loaded base model and model...")

Instructions for updating:
keep_dims is deprecated, use keepdims instead
[INFO] successfully loaded base model and model...


C:\deeplearning\anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("cu..., inputs=Tensor("in...)`


In [11]:
# path to training dataset
train_labels = os.listdir(train_path)

In [12]:
# Create a LabelEncoder object to encode the training labels
print ("[INFO] encoding labels...")
le = LabelEncoder()
le.fit([tl for tl in train_labels])

[INFO] encoding labels...


LabelEncoder()

In [13]:
# variables to hold features and labels
features = []
labels   = []

In [14]:
# loop over all the labels in the folder
count = 1
for i, label in enumerate(train_labels):
  cur_path = train_path + "/" + label
  count = 1
  for image_path in glob.glob(cur_path + "/*.jpg"):
    img = image.load_img(image_path, target_size=image_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature = model.predict(x)
    flat = feature.flatten()
    features.append(flat)
    labels.append(label)
    print ("[INFO] processed - " + str(count))
    count += 1
  print ("[INFO] completed label - " + label)

[INFO] processed - 1
[INFO] processed - 2
[INFO] processed - 3
[INFO] processed - 4
[INFO] processed - 5
[INFO] processed - 6
[INFO] processed - 7
[INFO] processed - 8
[INFO] processed - 9
[INFO] processed - 10
[INFO] processed - 11
[INFO] processed - 12
[INFO] processed - 13
[INFO] processed - 14
[INFO] processed - 15
[INFO] processed - 16
[INFO] processed - 17
[INFO] processed - 18
[INFO] processed - 19
[INFO] processed - 20
[INFO] processed - 21
[INFO] processed - 22
[INFO] processed - 23
[INFO] processed - 24
[INFO] processed - 25
[INFO] processed - 26
[INFO] processed - 27
[INFO] processed - 28
[INFO] processed - 29
[INFO] processed - 30
[INFO] processed - 31
[INFO] processed - 32
[INFO] processed - 33
[INFO] processed - 34
[INFO] processed - 35
[INFO] processed - 36
[INFO] processed - 37
[INFO] processed - 38
[INFO] processed - 39
[INFO] processed - 40
[INFO] processed - 41
[INFO] processed - 42
[INFO] processed - 43
[INFO] processed - 44
[INFO] processed - 45
[INFO] processed - 

[INFO] processed - 45
[INFO] processed - 46
[INFO] processed - 47
[INFO] processed - 48
[INFO] processed - 49
[INFO] processed - 50
[INFO] processed - 51
[INFO] processed - 52
[INFO] processed - 53
[INFO] processed - 54
[INFO] processed - 55
[INFO] processed - 56
[INFO] processed - 57
[INFO] processed - 58
[INFO] processed - 59
[INFO] processed - 60
[INFO] processed - 61
[INFO] processed - 62
[INFO] processed - 63
[INFO] processed - 64
[INFO] processed - 65
[INFO] processed - 66
[INFO] processed - 67
[INFO] processed - 68
[INFO] processed - 69
[INFO] processed - 70
[INFO] processed - 71
[INFO] processed - 72
[INFO] processed - 73
[INFO] processed - 74
[INFO] processed - 75
[INFO] processed - 76
[INFO] processed - 77
[INFO] processed - 78
[INFO] processed - 79
[INFO] processed - 80
[INFO] processed - 81
[INFO] processed - 82
[INFO] processed - 83
[INFO] processed - 84
[INFO] processed - 85
[INFO] processed - 86
[INFO] processed - 87
[INFO] processed - 88
[INFO] processed - 89
[INFO] pro

[INFO] processed - 405
[INFO] processed - 406
[INFO] processed - 407
[INFO] processed - 408
[INFO] processed - 409
[INFO] processed - 410
[INFO] processed - 411
[INFO] processed - 412
[INFO] processed - 413
[INFO] processed - 414
[INFO] processed - 415
[INFO] processed - 416
[INFO] processed - 417
[INFO] processed - 418
[INFO] processed - 419
[INFO] processed - 420
[INFO] processed - 421
[INFO] processed - 422
[INFO] processed - 423
[INFO] processed - 424
[INFO] processed - 425
[INFO] processed - 426
[INFO] processed - 427
[INFO] processed - 428
[INFO] processed - 429
[INFO] processed - 430
[INFO] processed - 431
[INFO] processed - 432
[INFO] processed - 433
[INFO] completed label - apple
[INFO] processed - 1
[INFO] processed - 2
[INFO] processed - 3
[INFO] processed - 4
[INFO] processed - 5
[INFO] processed - 6
[INFO] processed - 7
[INFO] processed - 8
[INFO] processed - 9
[INFO] processed - 10
[INFO] processed - 11
[INFO] processed - 12
[INFO] processed - 13
[INFO] processed - 14
[IN

[INFO] processed - 194
[INFO] processed - 195
[INFO] processed - 196
[INFO] processed - 197
[INFO] processed - 198
[INFO] processed - 199
[INFO] processed - 200
[INFO] processed - 201
[INFO] processed - 202
[INFO] processed - 203
[INFO] processed - 204
[INFO] processed - 205
[INFO] processed - 206
[INFO] processed - 207
[INFO] processed - 208
[INFO] processed - 209
[INFO] processed - 210
[INFO] processed - 211
[INFO] processed - 212
[INFO] processed - 213
[INFO] processed - 214
[INFO] processed - 215
[INFO] processed - 216
[INFO] processed - 217
[INFO] processed - 218
[INFO] processed - 219
[INFO] processed - 220
[INFO] processed - 221
[INFO] processed - 222
[INFO] processed - 223
[INFO] processed - 224
[INFO] processed - 225
[INFO] processed - 226
[INFO] processed - 227
[INFO] processed - 228
[INFO] processed - 229
[INFO] processed - 230
[INFO] processed - 231
[INFO] processed - 232
[INFO] processed - 233
[INFO] processed - 234
[INFO] processed - 235
[INFO] processed - 236
[INFO] proc

[INFO] processed - 53
[INFO] processed - 54
[INFO] processed - 55
[INFO] processed - 56
[INFO] processed - 57
[INFO] processed - 58
[INFO] processed - 59
[INFO] processed - 60
[INFO] processed - 61
[INFO] processed - 62
[INFO] processed - 63
[INFO] processed - 64
[INFO] processed - 65
[INFO] processed - 66
[INFO] processed - 67
[INFO] processed - 68
[INFO] processed - 69
[INFO] processed - 70
[INFO] processed - 71
[INFO] processed - 72
[INFO] processed - 73
[INFO] processed - 74
[INFO] processed - 75
[INFO] processed - 76
[INFO] processed - 77
[INFO] processed - 78
[INFO] processed - 79
[INFO] processed - 80
[INFO] processed - 81
[INFO] processed - 82
[INFO] processed - 83
[INFO] processed - 84
[INFO] processed - 85
[INFO] processed - 86
[INFO] processed - 87
[INFO] processed - 88
[INFO] processed - 89
[INFO] processed - 90
[INFO] processed - 91
[INFO] processed - 92
[INFO] processed - 93
[INFO] processed - 94
[INFO] processed - 95
[INFO] processed - 96
[INFO] processed - 97
[INFO] pro

...

[INFO] processed - 4
[INFO] processed - 5
[INFO] processed - 6
[INFO] processed - 7
[INFO] processed - 8
[INFO] processed - 9
[INFO] processed - 10
[INFO] processed - 11
[INFO] processed - 12
[INFO] processed - 13
[INFO] processed - 14
[INFO] processed - 15
[INFO] processed - 16
[INFO] processed - 17
[INFO] processed - 18
[INFO] processed - 19
[INFO] processed - 20
[INFO] processed - 21
[INFO] processed - 22
[INFO] processed - 23
[INFO] processed - 24
[INFO] processed - 25
[INFO] processed - 26
[INFO] processed - 27
[INFO] processed - 28
[INFO] processed - 29
[INFO] processed - 30
[INFO] processed - 31
[INFO] processed - 32
[INFO] processed - 33
[INFO] processed - 34
[INFO] processed - 35
[INFO] processed - 36
[INFO] processed - 37
[INFO] processed - 38
[INFO] processed - 39
[INFO] processed - 40
[INFO] processed - 41
[INFO] processed - 42
[INFO] processed - 43
[INFO] processed - 44
[INFO] processed - 45
[INFO] processed - 46
[INFO] processed - 47
[INFO] processed - 48
[INFO] processed

[INFO] processed - 77
[INFO] processed - 78
[INFO] processed - 79
[INFO] processed - 80
[INFO] processed - 81
[INFO] processed - 82
[INFO] processed - 83
[INFO] processed - 84
[INFO] processed - 85
[INFO] processed - 86
[INFO] processed - 87
[INFO] processed - 88
[INFO] processed - 89
[INFO] processed - 90
[INFO] processed - 91
[INFO] processed - 92
[INFO] processed - 93
[INFO] processed - 94
[INFO] processed - 95
[INFO] processed - 96
[INFO] processed - 97
[INFO] processed - 98
[INFO] processed - 99
[INFO] processed - 100
[INFO] processed - 101
[INFO] processed - 102
[INFO] processed - 103
[INFO] processed - 104
[INFO] processed - 105
[INFO] processed - 106
[INFO] processed - 107
[INFO] processed - 108
[INFO] processed - 109
[INFO] processed - 110
[INFO] processed - 111
[INFO] processed - 112
[INFO] processed - 113
[INFO] processed - 114
[INFO] processed - 115
[INFO] processed - 116
[INFO] processed - 117
[INFO] processed - 118
[INFO] processed - 119
[INFO] processed - 120
[INFO] proc

[INFO] processed - 436
[INFO] processed - 437
[INFO] processed - 438
[INFO] processed - 439
[INFO] processed - 440
[INFO] processed - 441
[INFO] processed - 442
[INFO] completed label - starbucks
[INFO] processed - 1
[INFO] processed - 2
[INFO] processed - 3
[INFO] processed - 4
[INFO] processed - 5
[INFO] processed - 6
[INFO] processed - 7
[INFO] processed - 8
[INFO] processed - 9
[INFO] processed - 10
[INFO] processed - 11
[INFO] processed - 12
[INFO] processed - 13
[INFO] processed - 14
[INFO] processed - 15
[INFO] processed - 16
[INFO] processed - 17
[INFO] processed - 18
[INFO] processed - 19
[INFO] processed - 20
[INFO] processed - 21
[INFO] processed - 22
[INFO] processed - 23
[INFO] processed - 24
[INFO] processed - 25
[INFO] processed - 26
[INFO] processed - 27
[INFO] processed - 28
[INFO] processed - 29
[INFO] processed - 30
[INFO] processed - 31
[INFO] processed - 32
[INFO] processed - 33
[INFO] processed - 34
[INFO] processed - 35
[INFO] processed - 36
[INFO] processed - 37

[INFO] processed - 98
[INFO] processed - 99
[INFO] processed - 100
[INFO] processed - 101
[INFO] processed - 102
[INFO] processed - 103
[INFO] processed - 104
[INFO] processed - 105
[INFO] processed - 106
[INFO] processed - 107
[INFO] processed - 108
[INFO] processed - 109
[INFO] processed - 110
[INFO] processed - 111
[INFO] processed - 112
[INFO] processed - 113
[INFO] processed - 114
[INFO] processed - 115
[INFO] processed - 116
[INFO] processed - 117
[INFO] processed - 118
[INFO] processed - 119
[INFO] processed - 120
[INFO] processed - 121
[INFO] processed - 122
[INFO] processed - 123
[INFO] processed - 124
[INFO] processed - 125
[INFO] processed - 126
[INFO] processed - 127
[INFO] processed - 128
[INFO] processed - 129
[INFO] processed - 130
[INFO] processed - 131
[INFO] processed - 132
[INFO] processed - 133
[INFO] processed - 134
[INFO] processed - 135
[INFO] processed - 136
[INFO] processed - 137
[INFO] processed - 138
[INFO] processed - 139
[INFO] processed - 140
[INFO] proces

[INFO] processed - 25
[INFO] processed - 26
[INFO] processed - 27
[INFO] processed - 28
[INFO] processed - 29
[INFO] processed - 30
[INFO] processed - 31
[INFO] processed - 32
[INFO] processed - 33
[INFO] processed - 34
[INFO] processed - 35
[INFO] processed - 36
[INFO] processed - 37
[INFO] processed - 38
[INFO] processed - 39
[INFO] processed - 40
[INFO] processed - 41
[INFO] processed - 42
[INFO] processed - 43
[INFO] processed - 44
[INFO] processed - 45
[INFO] processed - 46
[INFO] processed - 47
[INFO] processed - 48
[INFO] processed - 49
[INFO] processed - 50
[INFO] processed - 51
[INFO] processed - 52
[INFO] processed - 53
[INFO] processed - 54
[INFO] processed - 55
[INFO] processed - 56
[INFO] processed - 57
[INFO] processed - 58
[INFO] processed - 59
[INFO] processed - 60
[INFO] processed - 61
[INFO] processed - 62
[INFO] processed - 63
[INFO] processed - 64
[INFO] processed - 65
[INFO] processed - 66
[INFO] processed - 67
[INFO] processed - 68
[INFO] processed - 69
[INFO] pro

In [15]:
# encode the labels using LabelEncoder
le = LabelEncoder()
le_labels = le.fit_transform(labels)

In [16]:
# get the shape of training labels
print ("[STATUS] training labels: {}".format(le_labels))
print ("[STATUS] training labels shape: {}".format(le_labels.shape))

[STATUS] training labels: [ 1  1  1 ... 31 31 31]
[STATUS] training labels shape: (7830,)


In [17]:
# save features and labels
h5f_data = h5py.File(features_path, 'w')
h5f_data.create_dataset('dataset_1', data=np.array(features))

<HDF5 dataset "dataset_1": shape (7830, 131072), type "<f4">

In [18]:
h5f_label = h5py.File(labels_path, 'w')
h5f_label.create_dataset('dataset_1', data=np.array(le_labels))

<HDF5 dataset "dataset_1": shape (7830,), type "<i8">

In [19]:
h5f_data.close()
h5f_label.close()

In [20]:
h5f_data.close()
h5f_label.close()

In [22]:
# save model and weights
model_json = model.to_json()
with open(model_path + str(test_size) + ".json", "w") as json_file:
    json_file.write(model_json)

In [23]:
# save weights
model.save_weights(model_path + str(test_size) + ".h5")
print("[STATUS] saved model and weights to disk..")
print ("[STATUS] features and labels saved..")

[STATUS] saved model and weights to disk..
[STATUS] features and labels saved..


In [24]:
# end time
end = time.time()
print ("[STATUS] end time - {}".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")))

[STATUS] end time - 2018-09-14 00:21
